# <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/Escudo-UdeA.svg/620px-Escudo-UdeA.svg.png" width="100" align="left">

<h1 style="text-align: center;">  Componente Práctica - PDS 2023-2</h1>
<h2 style="text-align: center;"></h2>
<p style="text-align: center;"></p>

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.io.wavfile import read
from IPython.display import Audio
from matplotlib.pylab import *
from numpy import pi
import scipy.signal as sp 

## 1. Se desea diseñar un Filtro FIR rechaza-bandas por el método de enventanado, si se tienen las siguientes condiciones:
- Frecuencia de muestreo: 12 KHz
- Frecuencia de corte 1: 2KHz
- Frecuencia de corte 2: 4KHz
- Ripple: 0.3%
- Ancho de banda de la transición: 200 Hz

a) Qué ventana sería la más apropiada (entre Rectangular, Hanning, Hamming y Blackman) y cuál sería el tamaño del filtro FIR si se desea un N impar?

In [ ]:
#Frecuencia de corte en radianes

Fs = 12000
Fc1 = 2000
Fc2 = 4000
fm = 200

wc1 = (2*np.pi*Fc1)/Fs       #Frecuencia de corte 1 normalizada en radianes
wc2 = (2*np.pi*Fc2)/Fs       #Frecuencia de corte 2 normalizada en radianes
print("Frecuencia de corte 1 normalizada en radianes: ", wc1)
print("\nFrecuencia de corte 2 normalizada en radianes: ", wc2)

In [ ]:
#Riple en decibelios
ripple = 0.003
ripple_dB= 20*np.log10(ripple)
ripple_dB
print("Ripple en decibelios: ", ripple_dB)

Según el resultado, debemos usar una ventana Hamming para diseñar el filtro.
Se tiene la siguiente relación para M:

## Para la ventana Hann y Hamming:  $M\geq \frac{4F_s}{\Delta fm} $

In [ ]:
def getM(ripple_dB, Fs, fm):
    M = 0
    if (-53 > ripple_dB >= -74.0):
        print('Ventana Blackman')
        M = 6*Fs/fm        
    elif (-44 > ripple_dB >= -53.0):
        print('Ventana Hamming') 
        M = 4*Fs/fm           
    elif (-21 > ripple_dB >= -44.0):
        print('Ventana Hann')
        M = 4*Fs/fm        
    elif (ripple_dB  >= -21.0):
        print('Ventana Rectangular')
        M = 2*Fs/fm - 1       
    
    if M % 2 == 0:
        M += 1
    return int(M)

In [ ]:
M = getM( ripple_dB, Fs, fm)
print('Orden del filtro:', M)

b) Imponiendo una respuesta con simetría positiva y causal, halle h(n). Si el tamaño del filtro es muy grande, dar el valor de al menos los primeros 5 coeficientes, 5 coeficientes centrales y 5 coeficientes finales.

Como M se sugiere impar, se usa M= 241.

- Para Calcular los coeficientes del filtro, es necesario calcular primero los coeficientes de la respuesta al impulso del filtro ideal y la respuesta impulsional de la ventana.

In [ ]:
wc1 = 2*pi*Fc1/Fs                                     # frecuencia de corte normalizada en radianes
wc2 = 2*pi*Fc2/Fs                                     # frecuencia de corte normalizada en radianes
n = np.arange(-M//2+1, M//2+1)                        # Vector de muestras

hi = (wc1/pi) * np.sinc(wc1*n/pi) - (wc2/pi) * np.sinc(wc2*n/pi) # Respuesta del filtro ideal Rechaza-bandas
hi[M//2] = 1-(wc2-wc1)/pi
win = 0.54-0.46*np.cos(2*pi*np.arange(len(n))/(M-1))  # Ventana de Hamming
h_n = hi*win                                          # Multiplico la respuesta ideal por la ventana
w, h = sp.freqz(h_n, 1, whole=True, worN=2048)        # Respuesta en frecuencia del filtro enventanado

In [ ]:
# Grafica
plt.figure()
plt.title('Respuesta en frecuencia del rechaza-banda', fontsize=18)
plt.plot((w-pi)*Fs/(2*pi),np.abs(np.fft.fftshift(h)), label='$Filtro$')
plt.vlines(Fc1, 0, 1, color='r', lw=2, linestyle='--', label='$F_{c1}$')
plt.vlines(Fc2, 0, 1, color='r', lw=2, linestyle='--', label='$F_{c2}$')
plt.vlines(Fc1+fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c1}+f_m/2$')
plt.vlines(Fc1-fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c1}-f_m/2$')
plt.vlines(Fc2+fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c2}+f_m/2$')
plt.vlines(Fc2-fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c2}-f_m/2$')
plt.xlabel('Frecuencia (Hz)', fontsize=18)
plt.ylabel('Amplitud', fontsize=18)
plt.xlim([0, 6000])
plt.legend(loc='lower right')
plt.grid()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
axs[0].set_title('Respuesta en frecuencia del rechaza-banda')
axs[0].plot((w-pi)*Fs/(2*pi),np.abs(np.fft.fftshift(h)), label='$Filtro$')
axs[0].vlines(Fc1, 0, 1, color='r', lw=2, linestyle='--', label='$F_{c1}$')
axs[0].vlines(Fc2, 0, 1, color='r', lw=2, linestyle='--', label='$F_{c2}$')
axs[0].vlines(Fc1+fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c1}+f_m/2$')
axs[0].vlines(Fc1-fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c1}-f_m/2$')
axs[0].vlines(Fc2+fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c2}+f_m/2$')
axs[0].vlines(Fc2-fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c2}-f_m/2$')
axs[0].set_xlabel('Frecuencia (Hz)', fontsize=18)
axs[0].set_ylabel('Amplitud', fontsize=18)
axs[0].set_xlim([1800, 2200])
axs[0].grid()

axs[1].set_title('Respuesta en frecuencia del rechaza-banda')
axs[1].plot((w-pi)*Fs/(2*pi),np.abs(np.fft.fftshift(h)), label='$Filtro$')
axs[1].vlines(Fc1, 0, 1, color='r', lw=2, linestyle='--', label='$F_{c1}$')
axs[1].vlines(Fc2, 0, 1, color='r', lw=2, linestyle='--', label='$F_{c2}$')
axs[1].vlines(Fc1+fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c1}+f_m/2$')
axs[1].vlines(Fc1-fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c1}-f_m/2$')
axs[1].vlines(Fc2+fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c2}+f_m/2$')
axs[1].vlines(Fc2-fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c2}-f_m/2$')
axs[1].set_xlabel('Frecuencia (Hz)', fontsize=18)
axs[1].set_ylabel('Amplitud', fontsize=18)
axs[1].set_xlim([3800, 4200])
axs[1].grid()

plt.tight_layout()
plt.show()

In [ ]:
print("hi(0) = ",np.round(hi[M//2],6))

In [ ]:
hi_prime = {}  # Create an empty dictionary to store the transformed hi' values
for i in range(-M//2, M//2+1):
    hi_prime[i + M//2] = hi[i + M//2]  # Assign the corresponding hi value to hi' with adjusted index

# Print the hi' values
for i in range(M):
    original_index = i - M//2  # Calculate the original index
    print("hi'({}) = hi({}) =".format(i, original_index), np.round(hi_prime[i],6))

In [ ]:
win0 = 0.54-0.46*np.cos(2*pi*0/(M-1))  # Ventana de Hamming
print("win(0) = ",np.round(win0,6))

h_n_0 = hi_prime[0]*win0
print("h_n(0) = ", np.round(h_n_0,6)) 

In [ ]:
print('\nLos primeros 5 coeficientes')   
for i in np.arange(0,4):
    
    print("hi'({}) =".format(int(i)), np.round(hi_prime[i],6))
    
    win= 0.54-0.46*np.cos(2*pi*i/(M-1))  #Recordar reemplazar la M
    print("win ({})=".format(int(i)), np.round(win,6))

    h_n= hi_prime[i]*win
    print("h_n ({}) = ".format(int(i)), np.round(h_n,6),'\n')  

print('\nLos 5 coeficientes centrales')
for i in np.arange(M//2-2, M//2+3):
    
    print("hi'({}) =".format(int(i)), np.round(hi_prime[i],6))
    
    win= 0.54-0.46*np.cos(2*pi*i/(M-1))  #Recordar reemplazar la M
    print("win ({})=".format(int(i)), np.round(win,6))

    h_n= hi_prime[i]*win
    print("h_n ({}) = ".format(int(i)), np.round(h_n,6),'\n')

print('\nLos 5 finales') 
for i in np.arange(M-5, M):
    
    print("hi'({}) =".format(int(i)), np.round(hi_prime[i],6))
    
    win= 0.54-0.46*np.cos(2*pi*i/(M-1))  #Recordar reemplazar la M
    print("win ({})=".format(int(i)), np.round(win,6))

    h_n= hi_prime[i]*win
    print("h_n ({}) = ".format(int(i)), np.round(h_n,6),'\n')


Gráficas de la respuesta al impulso y el espectro resultante

In [ ]:
n = np.arange(-M//2,M//2)   #Vector de muestras
hi = +(wc1/np.pi)*np.sinc((wc1*n)/np.pi) - (wc2/np.pi)*np.sinc((wc2*n)/np.pi) #Respuesta del filtro ideal
hi[n==0] = 1-(wc2-wc1)/np.pi
win = 0.54-0.46*np.cos(2*np.pi*np.arange(len(n))/(M-1))  #Función ventana para eliminar el fenómeno de Gibbs
h2 = hi*win  # Multiplicación entre Respuesta ideal y la ventana

w1,Hh1 = sp.freqz(hi,1,whole=True, worN=2048) # Respuesta en frecuencia del filtro ideal

w2,Hh2 = sp.freqz(h2,1, whole=True, worN=2048)  #Respuesta en frecuencia del filtro enventanado

In [ ]:
fig1,axs1 = subplots(2,1)
fig1.set_size_inches((12,18))
subplots_adjust(hspace=0.4)

ax=axs1[0]
ax.stem(n+M,hi,basefmt='b-')
ax.set_title('Respuesta al impulso en frecuencia del filtro ideal')
ax.set_xlabel("$N$",fontsize=24)
ax.set_ylabel("$h_1$ (Amplitud)",fontsize=24)


ax=axs1[1]
ax.stem(n+M,h2,basefmt='b-')
ax.set_title('Respuesta en frecuencia de la ventana Hamming')
ax.set_xlabel("$N$",fontsize=24)
ax.set_ylabel("$h_2$",fontsize=24)

In [ ]:
deltafm= 200 #ancho de banda de transicion
plt.figure()

plt.plot((w1-np.pi)*Fs/(2*np.pi),np.abs(np.fft.fftshift(Hh1)))
plt.axis(xmax=Fs/2,xmin=-Fs/2)
plt.vlines(Fc1, 0, 1, color='r', lw=2, linestyle='--',label='$F_{c1}$')
plt.vlines(Fc1+deltafm/2, 0, 1, color='g', lw=2, linestyle='--',label='$F_{c1}+fm/2$')
plt.vlines(Fc1-deltafm/2, 0, 1, color='g', lw=2, linestyle='--',label='$F_{c1}-fm/2$')
plt.vlines(Fc2, 0, 1, color='r', lw=2, linestyle='--',label='$F_{c2}$')
plt.vlines(Fc1, 0, 1, color='r', lw=2, linestyle='--',label='$F_{c1}$')
plt.vlines(Fc2+deltafm/2, 0, 1, color='g', lw=2, linestyle='--',label='$F_{c2}+fm/2$')
plt.vlines(Fc2-deltafm/2, 0, 1, color='g', lw=2, linestyle='--',label='$F_{c2}-fm/2$')
plt.hlines(0,Fc1,Fc2,color='g',lw=2.,linestyle='--')
plt.title('Respuesta del filtro ideal')
plt.xlabel(r"$f (Hz)$",fontsize=18)
plt.ylabel(r"$|H(\omega)| $",fontsize=18)

In [ ]:
w2,Hh = sp.freqz(h2,1, whole=True, worN=2048) 
w2=w2[1:]
Hh=Hh[1:]

fig = plt.figure(figsize=(12,7))
plt.title('Frequency response')
plt.xlabel('Frequency [rad/sample]')
angles = np.unwrap(np.angle(Hh))
plt.plot(angles, 'g')
plt.ylabel('Angle (radians)', color='g')
plt.grid()
plt.axis('tight')
plt.show()

### Se debe diseñar un filtro pasa- altas por el metodo de muestreo en frecuencia con las siguientes caracteristicas:

- Banda de paso de 16 kHz
- Frecuencia de muestreo de 44 kHz
- N = 9

In [ ]:
Fs = 44.1e3
N = 9
if N % 2 == 0:
    kmax = N//2 - 1 # Por ser N par
else:
    kmax = (N-1)//2 # Por ser N impar
alpha = (N-1)/2 # Por formula
paso = Fs/N # Tamaño de paso para muestrear

In [ ]:
H_k = []
for k in range(kmax+1):
    if paso*k >=16e3 and paso*k <= 24e3:
        H_k.append(1)
    else:
        H_k.append(0)
H_k

In [ ]:
h_n = []
for n in range(N):
    suma = 0
    for k in np.arange(1, kmax+1):
        suma += (1/N)*2*H_k[k]*np.cos(2*pi*k*(n-alpha)/N)
    suma += (1/N)*H_k[0]
    h_n.append(suma)

In [ ]:
for it, item in enumerate(h_n):
    print(f'h[{it}] =', item)

In [ ]:
w, h = sp.freqz(h_n, 1, whole=True, worN=1024)
plt.figure()
plt.title('Filtro pasa-altas', fontsize=24)
plt.plot(w-pi, np.abs(np.fft.fftshift(h)), label='Filtro pasa-altas')
plt.vlines(2*pi*16e3/Fs, 0, 1.1, color='r', lw=2, linestyle='--', label='$F_c$')
plt.xlabel('Frecuencia (Hz)', fontsize=18)
plt.ylabel('Amplitud', fontsize=18)
plt.xlim([0, pi])
plt.legend(loc='best')
plt.show()

##  Filtro analógico Pasa-Banda y conviertalo a un filtro digital,  Filtro Butterworth Analógico De Orden 1 por apróximación en derivadas

In [ ]:
fc1 = 3e3   #Frecuencia de corte 1
fc2 = 5e3  #Frecuencia de corte 2
fs = 10*fc2 #Frecuencia de muestreo

In [ ]:
def zeropoles(num, den=1):
    w,h = sp.freqz(num,den)
    sys1 = sp.lti(num,den)
    ang = np.arange(0.0,2*np.pi, 0.01)
    xp = np.cos(ang)
    yp= np.sin(ang)
    plt.figure(figsize=(7,7))
    plt.plot(xp,yp, '--')
    plt.plot(sys1.zeros.real, sys1.zeros.imag, 'o')
    plt.plot(sys1.poles.real, sys1.poles.imag, 'x')
    plt.grid()
    plt.show()

In [ ]:
b,a = sp.butter(1,[fc1, fc2], 'bandpass', analog=True, output='ba', fs=None) #Filtro Pasa-Banda analógico
w,h = sp.freqs(b,a) #Vector de frecuencias
print('num='+str(b))
print('den='+str(a))

# Graficar
plt.figure(figsize=(15,7))
plt.semilogx(w, 20*np.log10(abs(h)), label='Filter')  #Gráfica usando escala logarítmica
plt.axvline(fc1, color='g', label='$f_{c1}$')
plt.axvline(fc2, color='g', label='$f_{c2}$')
plt.title('Butterworth filter Frequency Response', fontsize='18') 
plt.xlabel('Frequency [radians / second]',fontsize=18)
plt.ylabel('Amplitude [dB]',fontsize=18)
plt.margins(0, 0.01) #Márgenes de la gráfica
plt.grid(which='both', axis='both')
plt.legend()
plt.show() #Mostrar la gráfica

In [ ]:
import sympy as spy
from sympy import symbols, factor

# Crea símbolos
t, z = symbols('T, Z')

# Define una función transferencia
s = (1 - z**(-1)) / t

# Define la transferencia funcion Hz
Hz = (2000 * s) / (s**2 + 2e3 * s + 1.5e7)

# Simplifica y expande la transferencia funcion
simplified_Hz = spy.simplify(Hz)

# Agrupa términos semejantes de Z^2, Z y Z^0
simplified_Hz_grouped = spy.collect(simplified_Hz, z, evaluate=False)

# Imprime la función de transferencia simplificada y agrupada
print('H(Z) = ')
spy.pprint(simplified_Hz_grouped)

In [ ]:
T = 0.0001
b11 =[2000*T, -2000*T,0]  #Numerador 
a11 = [15000000*(T**2)+2000*T+1, -2000*T-2,1] #Denominador
wb, Hb11 = sp.freqz(b11, a11);  #Vector de frecuencias y respuesta del filtro

#Gráfica
plt.figure(figsize=(15,6))
plt.title('Filto Pasa-Banda', fontsize=18)
plt.plot((wb*fs)/(2*np.pi), np.abs(Hb11), label='Filtro')
plt.axvline(fc1, color='g', label='$f_{c1}$') #Frecuencia de corte 1
plt.axvline(fc2, color='g', label='$f_{c2}$') #Frecuencia de corte 2
plt.xlabel('$f(Hz)$', fontsize=18)
plt.ylabel('$|H(\omega)|$', fontsize=18)
plt.legend()
plt.grid()
plt.show()

#Diagrama de polos y ceros
zeropoles(b11, a11)

## Cargue 3 audios cortos y úselos como entrada de cada uno de los filtros. Realice un análisis espectral para cada caso.

Rediseño el filtro su frecuencia de muestreo para que este con la del audio 44100 HZ

In [ ]:
Fs3 = 44100
Fc1 = 2000
Fc2 = 4000
fm3 = 200

wc1 = (2*np.pi*Fc1)/Fs3       #Frecuencia de corte 1 normalizada en radianes
wc2 = (2*np.pi*Fc2)/Fs3       #Frecuencia de corte 2 normalizada en radianes
print("Frecuencia de corte 1 normalizada en radianes: ", wc1)
print("\nFrecuencia de corte 2 normalizada en radianes: ", wc2)

In [ ]:
#Ancho de banda de Trancisión requerida en radianes
deltafm3= 200 #ancho de banda de transicion

deltafmr3= (2*np.pi*deltafm3)/Fs3
print('\u0394fm:',deltafmr3, ' radianes')

In [ ]:
#Riple en decibelios
ripple = 0.003
ripple_dB= 20*np.log10(ripple)
ripple_dB
print("Ripple en decibelios: ", ripple_dB)

In [ ]:
M = getM( ripple_dB, Fs3, fm3)
print('Orden del filtro:', M)

In [ ]:
wc1 = 2*pi*Fc1/Fs3                                     # frecuencia de corte normalizada en radianes
wc2 = 2*pi*Fc2/Fs3                                     # frecuencia de corte normalizada en radianes
n = np.arange(-M//2+1, M//2+1)                        # Vector de muestras

hi = (wc1/pi) * np.sinc(wc1*n/pi) - (wc2/pi) * np.sinc(wc2*n/pi) # Respuesta del filtro ideal Rechaza-bandas
hi[M//2] = 1-(wc2-wc1)/pi
win = 0.54-0.46*np.cos(2*pi*np.arange(len(n))/(M-1))  # Ventana de Hamming
h_n3 = hi*win                                          # Multiplico la respuesta ideal por la ventana

w1,Hh1 = sp.freqz(hi,1,whole=True, worN=M) # Respuesta en frecuencia del filtro ideal

w, h = sp.freqz(h_n3, 1, whole=True, worN=2048)        # Respuesta en frecuencia del filtro enventanado

In [ ]:
print("hi(0) = ",np.round(hi[M//2],6))

In [ ]:
hi_prime = {}  # Create an empty dictionary to store the transformed hi' values
for i in range(-M//2, M//2+1):
    hi_prime[i + M//2] = hi[i + M//2]  # Assign the corresponding hi value to hi' with adjusted index

# Print the hi' values
for i in range(M):
    original_index = i - M//2  # Calculate the original index
    print("hi'({}) = hi({}) =".format(i, original_index), np.round(hi_prime[i],6))

In [ ]:
win0 = 0.54-0.46*np.cos(2*pi*0/(M-1))  # Ventana de Hamming
print("win(0) = ",np.round(win0,6))

h_n_0 = hi_prime[0]*win0
print("h_n(0) = ", np.round(h_n_0,6)) 

In [ ]:
print('\nLos primeros 5 coeficientes')   
for i in np.arange(0,4):
    
    print("hi'({}) =".format(int(i)), np.round(hi_prime[i],6))
    
    win= 0.54-0.46*np.cos(2*pi*i/(M-1))  #Recordar reemplazar la M
    print("win ({})=".format(int(i)), np.round(win,6))

    h_n= hi_prime[i]*win
    print("h_n ({}) = ".format(int(i)), np.round(h_n,6),'\n')  

print('\nLos 5 coeficientes centrales')
for i in np.arange(M//2-2, M//2+3):
    
    print("hi'({}) =".format(int(i)), np.round(hi_prime[i],6))
    
    win= 0.54-0.46*np.cos(2*pi*i/(M-1))  #Recordar reemplazar la M
    print("win ({})=".format(int(i)), np.round(win,6))

    h_n= hi_prime[i]*win
    print("h_n ({}) = ".format(int(i)), np.round(h_n,6),'\n')

print('\nLos 5 finales') 
for i in np.arange(M-5, M):
    
    print("hi'({}) =".format(int(i)), np.round(hi_prime[i],6))
    
    win= 0.54-0.46*np.cos(2*pi*i/(M-1))  #Recordar reemplazar la M
    print("win ({})=".format(int(i)), np.round(win,6))

    h_n= hi_prime[i]*win
    print("h_n ({}) = ".format(int(i)), np.round(h_n,6),'\n')

In [ ]:
plt.figure()

plt.plot((w1-np.pi)*Fs3/(2*np.pi),np.abs(np.fft.fftshift(Hh1)))
plt.axis(xmax=Fs3/2,xmin=-Fs3/2)
plt.vlines(Fc1, 0, 1, color='r', lw=2, linestyle='--',label='$F_{c1}$')
plt.vlines(Fc1+deltafm/2, 0, 1, color='g', lw=2, linestyle='--',label='$F_{c1}+fm/2$')
plt.vlines(Fc1-deltafm/2, 0, 1, color='g', lw=2, linestyle='--',label='$F_{c1}-fm/2$')
plt.vlines(Fc2, 0, 1, color='r', lw=2, linestyle='--',label='$F_{c2}$')
plt.vlines(Fc1, 0, 1, color='r', lw=2, linestyle='--',label='$F_{c1}$')
plt.vlines(Fc2+deltafm/2, 0, 1, color='g', lw=2, linestyle='--',label='$F_{c2}+fm/2$')
plt.vlines(Fc2-deltafm/2, 0, 1, color='g', lw=2, linestyle='--',label='$F_{c2}-fm/2$')
plt.hlines(0,Fc1,Fc2,color='g',lw=2.,linestyle='--')
plt.xlim([-7000,7000])
plt.title('Respuesta del filtro ideal')
plt.xlabel(r"$f (Hz)$",fontsize=18)
plt.ylabel(r"$|H(\omega)| $",fontsize=18)
plt.legend(loc='lower right')
plt.grid()
plt.show()

In [ ]:
# Grafica
plt.figure()
plt.title('Respuesta en frecuencia del rechaza-banda', fontsize=18)
plt.plot((w-pi)*Fs/(2*pi),np.abs(np.fft.fftshift(h)), label='$Filtro$')
plt.vlines(Fc1, 0, 1, color='r', lw=2, linestyle='--', label='$F_{c1}$')
plt.vlines(Fc2, 0, 1, color='r', lw=2, linestyle='--', label='$F_{c2}$')
plt.vlines(Fc1+fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c1}+f_m/2$')
plt.vlines(Fc1-fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c1}-f_m/2$')
plt.vlines(Fc2+fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c2}+f_m/2$')
plt.vlines(Fc2-fm/2, 0, 1, color='g', lw=2, linestyle='--', label='$F_{c2}-f_m/2$')
plt.xlabel('Frecuencia (Hz)', fontsize=18)
plt.ylabel('Amplitud', fontsize=18)
plt.xlim([0, 6000])
plt.legend(loc='lower right')
plt.grid()
plt.show()

### El pasa - altas queda como se diseñó 

##  El filtro digital Pasa-Banda a partir de un filtro analógico queda igual

In [ ]:
file_audio1 ='violin.wav'
fs1, x1 =read(file_audio1)
x1=x1/float(max(abs(x1))) # escala la amplitud de la senal
t1=np.arange(0, float(len(x1))/fs1, 1.0/fs1) # Vector de tiempo
plt.figure(figsize=(14,6))
plt.plot(t1,x1, color='darkorange') # Dibujar la grafica
# Los siguientes dos comandos dibujan las etiquetas de los ejes
plt.xlabel('Time (s)',fontsize=18) # Etiqueta eje X
plt.ylabel('Amplitude',fontsize=18) # Etiqueta eje Y
plt.title('Señal de audio 1',fontsize=18)

plt.show() # Mostrar la grafica
Audio(x1, rate=fs1) # para escuchar la senal, si se desea

In [ ]:
orig1 = np.fft.fft(x1)                            #FFT de la señal filtrada para mostrarla más comodamente
freq_orig1  = np.fft.fftfreq(orig1.size)*fs1       #Vector de Frecuencias

y1 = np.convolve(h_n3, x1, mode='same')      #Filtra la señal requerida
Y1 = np.fft.fft(y1)                          #FFT de la señal filtrada para mostrarla más comodamente
freq_Y1 = np.fft.fftfreq(Y1.size)*fs1        #Vector de Frecuencias

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(15, 18))

#Gráfica
axs[0].plot(freq_orig1, abs(orig1 )**2, label='$Espectro$')
axs[0].vlines(Fc1, 0, 1.03e6, color='r', lw=2, linestyle='--', label='$F_{c1}$')
axs[0].vlines(Fc1+deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c1}+f_m/2$')
axs[0].vlines(Fc1-deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c1}-f_m/2$')
axs[0].vlines(Fc2, 0, 1.03e6, color='r', lw=2, linestyle='--', label='$F_{c2}$')
axs[0].vlines(Fc2+deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c2}+f_m/2$')
axs[0].vlines(Fc2-deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c2}-f_m/2$')
axs[0].set_xlim([-6500,6500])
axs[0].set_xlabel("$Frecuencia [Hz]$", fontsize=18)
axs[0].set_ylabel("$|C_{k}|^2$", fontsize=18)
axs[0].set_title('Espectro de la señal original', fontsize=18)
axs[0].legend(loc='upper right')
axs[0].grid()

axs[1].plot(freq_Y1, abs(Y1)**2, label='$Espectro$')
axs[1].vlines(Fc1, 0, 32000, color='r', lw=2, linestyle='--', label='$F_{c1}$')
axs[1].vlines(Fc1+deltafm/2, 0, 32000, color='g', lw=2, linestyle='--', label='$F_{c1}+f_m/2$')
axs[1].vlines(Fc1-deltafm/2, 0, 32000, color='g', lw=2, linestyle='--', label='$F_{c1}-f_m/2$')
axs[1].vlines(Fc2, 0, 32000, color='r', lw=2, linestyle='--', label='$F_{c2}$')
axs[1].vlines(Fc2+deltafm/2, 0, 32000, color='g', lw=2, linestyle='--', label='$F_{c2}+f_m/2$')
axs[1].vlines(Fc2-deltafm/2, 0, 32000, color='g', lw=2, linestyle='--', label='$F_{c2}-f_m/2$')
axs[1].set_xlim([1500,4500])
axs[1].set_ylim([-1000,33000])
axs[1].set_xlabel("$Frecuencia [Hz]$", fontsize=18)
axs[1].set_ylabel("$|C_{k}|^2$", fontsize=18)
axs[1].set_title('Espectro de la señal filtrada', fontsize=18)
axs[1].legend(loc='upper right')
axs[1].grid()


#Gráfica
axs[2].plot(freq_Y1, abs(Y1)**2, label='$Espectro$')
axs[2].vlines(Fc1, 0, 1.03e6, color='r', lw=2, linestyle='--', label='$F_{c1}$')
axs[2].vlines(Fc1+deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c1}+f_m/2$')
axs[2].vlines(Fc1-deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c1}-f_m/2$')
axs[2].vlines(Fc2, 0, 1.03e6, color='r', lw=2, linestyle='--', label='$F_{c2}$')
axs[2].vlines(Fc2+deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c2}+f_m/2$')
axs[2].vlines(Fc2-deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c2}-f_m/2$')
axs[2].set_xlim([-6500,6500])
axs[2].set_xlabel("$Frecuencia [Hz]$", fontsize=18)
axs[2].set_ylabel("$|C_{k}|^2$", fontsize=18)
axs[2].set_title('Espectro de la señal filtrada, vista completa', fontsize=18)
axs[2].legend(loc='upper right')
axs[2].grid()
plt.tight_layout()
plt.show()

In [ ]:
y1 =y1/float(max(abs(y1))) # escala la amplitud de la senal
t1=np.arange(0, float(len(y1 ))/fs1, 1.0/fs1) # Vector de tiempo
plt.figure(figsize=(14,6))
plt.plot(t1,x1, color='darkorange', label='Original') # Dibujar la grafica
plt.plot(t1,y1 , color='darkgreen',label='Filtrada') # Dibujar la grafica
# Los siguientes dos comandos dibujan las etiquetas de los ejes
plt.xlabel('Time (s)',fontsize=18) # Etiqueta eje X
plt.ylabel('Amplitude',fontsize=18) # Etiqueta eje Y
plt.title('Señales de Audio (Original y Filtrada)',fontsize=18)
plt.legend(loc='best')

plt.show() # Mostrar la grafica
Audio(y1 , rate=fs1) #

In [ ]:
file_audio2 ='0-9_Male_Vocalized.wav'
fs2, x2 =read(file_audio2)
x2=x2/float(max(abs(x2))) # escala la amplitud de la senal
t2=np.arange(0, float(len(x2))/fs2, 1.0/fs2) # Vector de tiempo
plt.figure(figsize=(14,6))
plt.plot(t2,x2, color='y') # Dibujar la grafica
# Los siguientes dos comandos dibujan las etiquetas de los ejes
plt.xlabel('Time (s)',fontsize=18) # Etiqueta eje X
plt.ylabel('Amplitude',fontsize=18) # Etiqueta eje Y
plt.title('Señal de audio 2',fontsize=18)

plt.show() # Mostrar la grafica
Audio(x2, rate=fs2) # para escuchar la senal, si se desea

In [ ]:
orig2 = np.fft.fft(x2)                            #FFT de la señal filtrada para mostrarla más comodamente
freq_orig2  = np.fft.fftfreq(orig2.size)*fs2       #Vector de Frecuencias

y2 = np.convolve(h_n3, x2, mode='same')      #Filtra la señal requerida
Y2 = np.fft.fft(y2)                          #FFT de la señal filtrada para mostrarla más comodamente
freq_Y2 = np.fft.fftfreq(Y2.size)*fs2        #Vector de Frecuencias

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(15, 18))

#Gráfica
axs[0].plot(freq_orig2, abs(orig2)**2, label='$Espectro$')
axs[0].vlines(Fc1, 0, 1.03e6, color='r', lw=2, linestyle='--', label='$F_{c1}$')
axs[0].vlines(Fc1+deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c1}+f_m/2$')
axs[0].vlines(Fc1-deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c1}-f_m/2$')
axs[0].vlines(Fc2, 0, 1.03e6, color='r', lw=2, linestyle='--', label='$F_{c2}$')
axs[0].vlines(Fc2+deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c2}+f_m/2$')
axs[0].vlines(Fc2-deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c2}-f_m/2$')
axs[0].set_xlim([-6500,6500])
axs[0].set_xlabel("$Frecuencia [Hz]$", fontsize=18)
axs[0].set_ylabel("$|C_{k}|^2$", fontsize=18)
axs[0].set_title('Espectro de la señal original', fontsize=18)
axs[0].legend(loc='upper right')
axs[0].grid()

axs[1].plot(freq_Y2, abs(Y2)**2, label='$Espectro$')
axs[1].vlines(Fc1, 0, 32000, color='r', lw=2, linestyle='--', label='$F_{c1}$')
axs[1].vlines(Fc1+deltafm/2, 0, 32000, color='g', lw=2, linestyle='--', label='$F_{c1}+f_m/2$')
axs[1].vlines(Fc1-deltafm/2, 0, 32000, color='g', lw=2, linestyle='--', label='$F_{c1}-f_m/2$')
axs[1].vlines(Fc2, 0, 32000, color='r', lw=2, linestyle='--', label='$F_{c2}$')
axs[1].vlines(Fc2+deltafm/2, 0, 32000, color='g', lw=2, linestyle='--', label='$F_{c2}+f_m/2$')
axs[1].vlines(Fc2-deltafm/2, 0, 32000, color='g', lw=2, linestyle='--', label='$F_{c2}-f_m/2$')
axs[1].set_xlim([1500,4500])
axs[1].set_ylim([-1000,33000])
axs[1].set_xlabel("$Frecuencia [Hz]$", fontsize=18)
axs[1].set_ylabel("$|C_{k}|^2$", fontsize=18)
axs[1].set_title('Espectro de la señal filtrada', fontsize=18)
axs[1].legend(loc='upper right')
axs[1].grid()


#Gráfica
axs[2].plot(freq_Y2, abs(Y2)**2, label='$Espectro$')
axs[2].vlines(Fc1, 0, 1.03e6, color='r', lw=2, linestyle='--', label='$F_{c1}$')
axs[2].vlines(Fc1+deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c1}+f_m/2$')
axs[2].vlines(Fc1-deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c1}-f_m/2$')
axs[2].vlines(Fc2, 0, 1.03e6, color='r', lw=2, linestyle='--', label='$F_{c2}$')
axs[2].vlines(Fc2+deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c2}+f_m/2$')
axs[2].vlines(Fc2-deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c2}-f_m/2$')
axs[2].set_xlim([-6500,6500])
axs[2].set_xlabel("$Frecuencia [Hz]$", fontsize=18)
axs[2].set_ylabel("$|C_{k}|^2$", fontsize=18)
axs[2].set_title('Espectro de la señal filtrada, vista completa', fontsize=18)
axs[2].legend(loc='upper right')
axs[2].grid()
plt.tight_layout()
plt.show()

In [ ]:
y2 =y2/float(max(abs(y2))) # escala la amplitud de la senal
t2=np.arange(0, float(len(y2))/fs2, 1.0/fs2) # Vector de tiempo
plt.figure(figsize=(14,6))
plt.plot(t2,x2, color='darkorange', label='Original') # Dibujar la grafica
plt.plot(t2,y2 , color='darkgreen',label='Filtrada') # Dibujar la grafica
# Los siguientes dos comandos dibujan las etiquetas de los ejes
plt.xlabel('Time (s)',fontsize=18) # Etiqueta eje X
plt.ylabel('Amplitude',fontsize=18) # Etiqueta eje Y
plt.title('Señales de Audio (Original y Filtrada)',fontsize=18)
plt.legend(loc='best')

plt.show() # Mostrar la grafica
Audio(y2 , rate=fs2) #

In [ ]:
file_audio3 ='Sounds_of_War.wav'
fs3, x3 =read(file_audio3)
x3=x3/float(max(abs(x3))) # escala la amplitud de la senal
t3=np.arange(0, float(len(x3))/fs3, 1.0/fs3) # Vector de tiempo
plt.figure(figsize=(14,6))
plt.plot(t3,x3, color='brown') # Dibujar la grafica
# Los siguientes dos comandos dibujan las etiquetas de los ejes
plt.xlabel('Time (s)',fontsize=18) # Etiqueta eje X
plt.ylabel('Amplitude',fontsize=18) # Etiqueta eje Y
plt.title('Señal de audio 3 (Normalizada)',fontsize=18)

plt.show() # Mostrar la grafica
Audio(x3, rate=fs3) # para escuchar la senal, si se desea

In [ ]:
orig3 = np.fft.fft(x3)                            #FFT de la señal filtrada para mostrarla más comodamente
freq_orig3  = np.fft.fftfreq(orig3.size)*fs3       #Vector de Frecuencias

y3 = np.convolve(h_n3, x3, mode='same')      #Filtra la señal requerida
Y3 = np.fft.fft(y3)                          #FFT de la señal filtrada para mostrarla más comodamente
freq_Y3 = np.fft.fftfreq(Y3.size)*fs3        #Vector de Frecuencias

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(15, 18))

#Gráfica
axs[0].plot(freq_orig3, abs(orig3 )**2, label='$Espectro$')
axs[0].vlines(Fc1, 0, 1.03e6, color='r', lw=2, linestyle='--', label='$F_{c1}$')
axs[0].vlines(Fc1+deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c1}+f_m/2$')
axs[0].vlines(Fc1-deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c1}-f_m/2$')
axs[0].vlines(Fc2, 0, 1.03e6, color='r', lw=2, linestyle='--', label='$F_{c2}$')
axs[0].vlines(Fc2+deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c2}+f_m/2$')
axs[0].vlines(Fc2-deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c2}-f_m/2$')
axs[0].set_xlim([-6500,6500])
axs[0].set_xlabel("$Frecuencia [Hz]$", fontsize=18)
axs[0].set_ylabel("$|C_{k}|^2$", fontsize=18)
axs[0].set_title('Espectro de la señal original', fontsize=18)
axs[0].legend(loc='upper right')
axs[0].grid()

axs[1].plot(freq_Y3, abs(Y3)**2, label='$Espectro$')
axs[1].vlines(Fc1, 0, 32000, color='r', lw=2, linestyle='--', label='$F_{c1}$')
axs[1].vlines(Fc1+deltafm/2, 0, 32000, color='g', lw=2, linestyle='--', label='$F_{c1}+f_m/2$')
axs[1].vlines(Fc1-deltafm/2, 0, 32000, color='g', lw=2, linestyle='--', label='$F_{c1}-f_m/2$')
axs[1].vlines(Fc2, 0, 32000, color='r', lw=2, linestyle='--', label='$F_{c2}$')
axs[1].vlines(Fc2+deltafm/2, 0, 32000, color='g', lw=2, linestyle='--', label='$F_{c2}+f_m/2$')
axs[1].vlines(Fc2-deltafm/2, 0, 32000, color='g', lw=2, linestyle='--', label='$F_{c2}-f_m/2$')
axs[1].set_xlim([1500,4500])
axs[1].set_ylim([-1000,33000])
axs[1].set_xlabel("$Frecuencia [Hz]$", fontsize=18)
axs[1].set_ylabel("$|C_{k}|^2$", fontsize=18)
axs[1].set_title('Espectro de la señal filtrada', fontsize=18)
axs[1].legend(loc='upper right')
axs[1].grid()


#Gráfica
axs[2].plot(freq_Y3, abs(Y3)**2, label='$Espectro$')
axs[2].vlines(Fc1, 0, 1.03e6, color='r', lw=2, linestyle='--', label='$F_{c1}$')
axs[2].vlines(Fc1+deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c1}+f_m/2$')
axs[2].vlines(Fc1-deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c1}-f_m/2$')
axs[2].vlines(Fc2, 0, 1.03e6, color='r', lw=2, linestyle='--', label='$F_{c2}$')
axs[2].vlines(Fc2+deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c2}+f_m/2$')
axs[2].vlines(Fc2-deltafm/2, 0, 1.03e6, color='g', lw=2, linestyle='--', label='$F_{c2}-f_m/2$')
axs[2].set_xlim([-6500,6500])
axs[2].set_xlabel("$Frecuencia [Hz]$", fontsize=18)
axs[2].set_ylabel("$|C_{k}|^2$", fontsize=18)
axs[2].set_title('Espectro de la señal filtrada, vista completa', fontsize=18)
axs[2].legend(loc='upper right')
axs[2].grid()
plt.tight_layout()
plt.show()

In [ ]:
y3 =y3/float(max(abs(y3))) # escala la amplitud de la senal
t3=np.arange(0, float(len(y3))/fs3, 1.0/fs3) # Vector de tiempo
plt.figure(figsize=(14,6))
plt.plot(t3,x3, color='darkorange', label='Original') # Dibujar la grafica
plt.plot(t3,y3 , color='darkgreen',label='Filtrada') # Dibujar la grafica
# Los siguientes dos comandos dibujan las etiquetas de los ejes
plt.xlabel('Time (s)',fontsize=18) # Etiqueta eje X
plt.ylabel('Amplitude',fontsize=18) # Etiqueta eje Y
plt.title('Señales de Audio (Original y Filtrada)',fontsize=18)
plt.legend(loc='best')

plt.show() # Mostrar la grafica
Audio(y3 , rate=fs3) #

## 2. Ecualizacion

In [ ]:
import numpy as np  
from scipy.io import wavfile
import matplotlib.pyplot as plt
from scipy.signal import freqz
import sounddevice as sd
from numpy import pi

# Cargar audio
samplerate, audio = wavfile.read('violin.wav')  

# Frecuencias de corte iniciales
fc_low = [200, 500, 2000, 6000, 8000]
fc_high = [500, 1000, 5000, 8000, 15000]

# Ganancias por banda  
gains = [1, 5, 1, -3, 1]  

# Diseño de filtros FIR y ventanas
filts = []
for i, (flow, fhigh, gain) in enumerate(zip(fc_low, fc_high, gains)):

    if i == 0:  # Banda 1 - Filtro pasa bajos con ventana blackman
        Nlow = 2646
        nlow = np.arange(-Nlow//2+1, Nlow//2+1)
        w1 = 2 * pi * flow / (samplerate)
        hi1 = w1 / pi * np.sinc(w1 * nlow / pi)
        win1 = 0.42 - 0.5 * np.cos(2 * pi * nlow / (Nlow - 1)) + 0.08 * np.cos(4 * pi * nlow / (Nlow - 1))
        fir_low = hi1 * win1
        filts.append(('low', fir_low, gain))

    elif i == 4:  # Banda 5 - Filtro pasa altos con ventana blackman
        Nhigh = 2646 
        nhigh = np.arange(-Nhigh//2+1, Nhigh//2+1)
        w2 = 2 * pi * fhigh / (samplerate)
        hi2 = -(w2 / pi * np.sinc(w2 * nhigh / pi))
        hi2[Nhigh//2] = 1 - 2 * (fhigh / samplerate)
        win2 = 0.42 - 0.5 * np.cos(2 * pi * nhigh / (Nhigh - 1)) + 0.08 * np.cos(4 * pi * nhigh / (Nhigh - 1))
        fir_high = hi2 * win2
        filts.append(('high', fir_high, gain))

    else:  # Banda 2, 3, 4 - Filtro pasa banda con ventana hamming
        Nmed = 1764
        nmed = np.arange(-Nmed//2+1, Nmed//2+1)
        w_low = 2 * pi * flow / (samplerate)
        w_high = 2 * pi * fhigh / (samplerate)
        hi3 = (w_high / pi) * np.sinc(w_high * nmed / pi) - (w_low / pi) * np.sinc(w_low * nmed / pi)
        hi3[Nmed//2] = 1 - (w_high - w_low) / pi
        win3 = 0.54 - 0.46 * np.cos(2 * pi * np.arange(len(nmed)) / (Nmed - 1))
        fir_bp = hi3 * win3
        filts.append(('bp', fir_bp, gain))

# Graficar respuesta en frecuencia para cada filtro
plt.figure(figsize=(12, 10))

for i, (type_, fir, gain) in enumerate(filts):
    freq, resp = freqz(fir * gain, whole=True, worN=8000, fs=samplerate)
    plt.plot(freq, 20 * np.log10(np.abs(resp)), label=f'{type_} - Banda {i+1}, Ganancia: {gain} dB')

plt.title('Respuesta en Frecuencia de los Filtros')
plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Amplitud (dB)')
plt.legend()
plt.grid(True)
plt.show()

# Filtrado de cada banda  
audio_eq = np.zeros_like(audio, dtype="float64")
for type_, fir, gain in filts:
    audio_eq += np.convolve(audio, fir * gain)[:len(audio)]

# Normalizar la señal para evitar clipping
audio_eq /= np.max(np.abs(audio_eq))

audio_norm = audio / np.max(np.abs(audio))
audio_eq_norm = audio_eq / np.max(np.abs(audio_eq)) 

# Señal original
plt.figure(figsize=(10, 5))
plt.plot(audio_norm, color='darkorange', label='Señal Original')
plt.legend()
plt.xlabel('Muestras')
plt.ylabel('Amplitud Normalizada')  
plt.title('Señal Original')
plt.grid(True)
plt.show()

# Señal ecualizada
plt.figure(figsize=(10, 5)) 
plt.plot(audio_eq_norm, color='darkred', label='Señal Ecualizada')
plt.legend()
plt.xlabel('Muestras')
plt.ylabel('Amplitud Normalizada')
plt.title('Señal Ecualizada')
plt.grid(True)
plt.show()

# Comparación
plt.figure(figsize=(10, 5))
plt.plot(audio_norm, color='darkorange', label='Original')
plt.plot(audio_eq_norm, color='darkred', label='Ecualizada') 
plt.legend()
plt.xlabel('Muestras')
plt.ylabel('Amplitud Normalizada')  
plt.title('Comparación')
plt.grid(True)
plt.show()

# Reproducir audio ecualizado
sd.play(audio_eq, samplerate)
sd.wait()
